# Big Data Dashboard

In [1]:
import seaborn as sns
from ipywidgets import interact

sns.set(style="darkgrid")

In [2]:
tips = sns.load_dataset('tips')

In [3]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
@interact(hue=['smoker', 'sex', 'time', 'day'])
def plot(hue):
    _ = sns.pairplot(tips, hue=hue)

interactive(children=(Dropdown(description='hue', options=('smoker', 'sex', 'time', 'day'), value='smoker'), O…

In [34]:
import pymongo
import pandas as pd

In [35]:
DATABASE_NAME = "big_data"
STAT_TABLE = "statistics"
SLOT_TABLE = "timeslots"
WINDOW_TABLE = "window"

In [36]:
mongo_client = pymongo.MongoClient("mongodb://localhost:27017")
used_db = mongo_client[DATABASE_NAME]

statistics = used_db[STAT_TABLE]
timeslots = used_db[SLOT_TABLE]
window = used_db[WINDOW_TABLE]

In [40]:
#my_query = statistics.find({"sensor_type": sensor_type})
my_query = statistics.find({"measurement": {'$gt': 0}})
res = pd.DataFrame(my_query)
res.head()

,_id,measurement,municipality,p-i,sensor_type,time,voltage
0,5ccb34aa56cf0b8e3b605392,17.636400,1081,41-0,0,2017-02-28 00:00:00.264,2.65796
1,5ccb34aa56cf0b8e3b605393,16.695601,1000,44-0,0,2017-02-28 00:00:00.271,2.66964
2,5ccb34aa56cf0b8e3b605394,19.145599,1160,33-0,0,2017-02-28 00:00:18.298,2.69742
3,5ccb34aa56cf0b8e3b605395,17.126801,1200,53-0,0,2017-02-28 00:00:18.552,2.74143
4,5ccb34aa56cf0b8e3b605396,18.910400,1040,19-0,0,2017-02-28 00:00:18.562,2.74532


## Basic statistics

In [174]:
from datetime import *
import time
import matplotlib.pyplot as plt

In [194]:
#my_query = statistics.find({"max": {'$gt': 0}})
#my_query = [elem for elem in my_query if "type" in elem.keys()]
#a_set = pd.DataFrame(my_query)
#a_set.head()

day = timedelta(days=1)

# query 1: stats (min, max, avg) per type (hue 1) per space (hue 2) per time (hue 3)
@interact(sensor_type = [("temperature", 0), ("humidity",1), ("light",2), ("movement",3)], 
          space_type = ["space", "municipality", "Brussels"],
          period = [("24hrs", day), ("2 days", 2 * day), ("last week", 7 * day), ("last month", 30*day), ("last year", 365*day)])

def report_statistics(sensor_type, space_type, period):
    
    # define current date
    current_date = datetime.strptime("2017-02-28 00:01:19.196", "%Y-%m-%d %H:%M:%S.%f")
    
    # query to obtain sensor type between current_time - period to now
    my_query = statistics.find({"sensor_type": sensor_type, "time": {'$gt': current_date - period}})
    res = pd.DataFrame(my_query)
    
    # transform information in DF to solely about IDs
    res["p-i"] = res["p-i"].map(lambda x: int(x.rstrip('0123').rstrip("-")))
    
    # case: space
    if space_type == "space":
        list_max = res.groupby(["p-i"])["measurement"].max()
        list_min = res.groupby(["p-i"])["measurement"].min()
        list_mean = res.groupby(["p-i"])["measurement"].mean()
        
        spaces = list(res.groupby(["p-i"]).groups.keys())
        max_values, min_values, mean_values = list(list_max.values), list(list_min.values), list(list_mean.values)

        print("space\tmax\t\t\tmin\t\t\tavg")
        for i in range(len(spaces)):
            print("{}\t{}\t{}\t{}".format(spaces[i], max_values[i], min_values[i], mean_values[i]))
        
    # case : municipality
    elif space_type == "municipality":
        
        list_max = res.groupby(["municipality"])["measurement"].max()
        list_min = res.groupby(["municipality"])["measurement"].min()
        list_mean = res.groupby(["municipality"])["measurement"].mean()
        
        municipalities = list(res.groupby(["municipality"]).groups.keys())
        max_values, min_values, mean_values = list(list_max.values), list(list_min.values), list(list_mean.values)
    
        # fixed number -> can be plotted
        plt.xlabel("municipalities")
        plt.xticks(rotation=90)
        plt.ylabel("reading")
        plt.plot(municipalities, max_values)
        plt.plot(municipalities, min_values)
        plt.plot(municipalities, mean_values)
        plt.show()
        
        print("municipality\tmax\t\t\tmin\t\t\tavg")
        for i in range(len(municipalities)):
            print("{}\t\t{}\t{}\t{}".format(municipalities[i], max_values[i], min_values[i], mean_values[i]))
    
    # case : Brussels  
    else:
        
        maximum_measurement = res["measurement"].max()
        minimum_measurement = res["measurement"].min()
        average_measurement = res["measurement"].mean()
        
        print("max_value : {}".format(maximum_measurement))
        print("min_value : {}".format(minimum_measurement))
        print("avg_value : {}".format(average_measurement))

        



interactive(children=(Dropdown(description='sensor_type', options=(('temperature', 0), ('humidity', 1), ('ligh…

## @interact(hue=["min", "max", "avg"])
def plot(hue):
    _ = sns.relplot(x="type", y=hue, data=a_set)

## Timeslots

## Sliding window